![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/arc/images/microsoft-small-logo.png)
 
## Create a PostgreSQL Hyperscale - Azure Arc on an existing Azure Arc Data Controller
 
This notebook walks through the process of creating a PostgreSQL Hyperscale - Azure Arc on an existing Azure Arc Data Controller.
 
* Follow the instructions in the **Prerequisites** cell to install the tools if not already installed.
* Make sure you have the target Azure Arc Data Controller already created.

<span style="color:red"><font size="3">Please press the "Run All" button to run the notebook</font></span>

### **Prerequisites** 
Ensure the following tools are installed and added to PATH before proceeding.
 
|Tools|Description|Installation|
|---|---|---|
|Azure CLI (az) | Command-line tool for installing and managing resources in an Azure Arc cluster |[Installation](https://docs.microsoft.com/cli/azure/install-azure-cli-windows?tabs=azure-cli) |
|Azure CLI arcdata extension | Commands for using Azure Arc for Azure data services. | [Installation](https://docs.microsoft.com/azure/azure-arc/data/install-arcdata-extension)

### **Setup and Check Prerequisites**

In [ ]:
import sys,os,json,subprocess
def run_command():
    print("Executing: " + cmd)
    output = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True )
    if output.returncode != 0:
        print(f'Command: {cmd} failed \n')
        print(f'\t>>>Error output: {output.stderr.decode("utf-8")}\n')
        sys.exit(f'exit code: {output.returncode}\n')
    print(f'Successfully executed: {cmd}')
    print(f'\t>>>Output: {output.stdout.decode("utf-8")}\n')
    return output.stdout.decode("utf-8")
cmd = 'az --version'
out = run_command()


### **Set variables**

#### 

Generated by Azure Data Studio using the values collected in the 'Deploy PostgreSQL Hyperscale - Azure Arc instance' wizard

### **Creating the PostgreSQL Hyperscale - Azure Arc instance**

In [ ]:
print (f'Creating the PostgreSQL Hyperscale - Azure Arc instance')

workers = f' --workers {postgres_server_group_workers}' if postgres_server_group_workers else ""
port = f' --port "{postgres_server_group_port}"' if postgres_server_group_port else ""
engine_version = f' --engine-version {postgres_server_group_engine_version}' if postgres_server_group_engine_version else ""
extensions = f' --extensions "{postgres_server_group_extensions}"' if postgres_server_group_extensions else ""
volume_size_data = f' --volume-size-data {postgres_server_group_volume_size_data}Gi' if postgres_server_group_volume_size_data else ""
volume_size_logs = f' --volume-size-logs {postgres_server_group_volume_size_logs}Gi' if postgres_server_group_volume_size_logs else ""
volume_size_backups = f' --volume-size-backups {postgres_server_group_volume_size_backups}Gi' if postgres_server_group_volume_size_backups else ""

def get_per_role_argument(argument, roles, unit=''):
    value = ','.join(f'{role}={value}{unit}' for role,value in roles.items() if value not in (None, ''))
    return f' {argument} {value}' if value else ''

cores_request = get_per_role_argument('--cores-request', {
    'c': postgres_server_group_coordinator_cores_request,
    'w': postgres_server_group_workers_cores_request
})

cores_limit = get_per_role_argument('--cores-limit', {
    'c': postgres_server_group_coordinator_cores_limit,
    'w': postgres_server_group_workers_cores_limit
})

memory_request = get_per_role_argument('--memory-request', {
    'c': postgres_server_group_coordinator_memory_request,
    'w': postgres_server_group_workers_memory_request
}, 'Gi')

memory_limit = get_per_role_argument('--memory-limit', {
    'c': postgres_server_group_coordinator_memory_limit,
    'w': postgres_server_group_workers_memory_limit
}, 'Gi')

os.environ["AZDATA_PASSWORD"] = os.environ["AZDATA_NB_VAR_POSTGRES_SERVER_GROUP_PASSWORD"]
cmd = f'az postgres arc-server create --name {postgres_server_group_name} --k8s-namespace {arc_data_controller_namespace} --use-k8s --storage-class-data {postgres_storage_class_data} --storage-class-logs {postgres_storage_class_logs} --storage-class-backups {postgres_storage_class_backups}{workers}{port}{engine_version}{extensions}{volume_size_data}{volume_size_logs}{volume_size_backups}{cores_request}{cores_limit}{memory_request}{memory_limit}'
out=run_command()